##### Parte 1/3: Subida de datos a la nube, la cual nos permitira alamcenar la BigData extraida de la API de Compras Públicas

# PROYECTO PARA IACKATHON DE COMPRAS PUBLICAS

## PASO 1:
Este código es una solución efectiva y altamente escalable para la recopilación y almacenamiento de datos públicos de OCIDs (Identificadores de Contratos de Obra) de compras públicas en Ecuador. Su robustez radica en varios aspectos clave. 
- Primero, automatiza la recopilación de datos de manera programática, eliminando la necesidad de realizar esta tarea manualmente, lo que ahorra tiempo y recursos significativos. 
- Segundo, se integra con Google Cloud Storage, lo que garantiza un almacenamiento seguro y confiable en la nube, facilitando el acceso y la gestión de datos a largo plazo. Además, su diseño modular permite una escalabilidad sencilla: se pueden agregar más años de datos simplemente configurando la variable year_to_retrieve. 

Además, es altamente personalizable para la ejecución programada, ya sea mensual o quimestralmente, mediante la configuración de tareas cron o programación de trabajos periódicos, lo que asegura la actualización continua de los datos y su disponibilidad para análisis y toma de decisiones en curso. En resumen, este código ofrece una solución versátil y automatizada para la gestión de datos públicos en el contexto de compras públicas en Ecuador.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Importar las bibliotecas necesarias
import requests  # Importa la biblioteca para realizar solicitudes HTTP
import pandas as pd  # Importa la biblioteca pandas para trabajar con datos tabulares
from google.cloud import storage  # Importa la biblioteca de Google Cloud Storage para almacenamiento en la nube

# Definir una función para obtener OCIDs de un año específico y una página específica
def get_ocids(year, page):
    # Construir la URL de búsqueda de OCIDs con el año y la página especificados
    ocid_search_url = f"https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/search_ocds?page={page}&year={year}"
    
    # Realizar una solicitud HTTP GET a la URL
    response = requests.get(ocid_search_url)
    
    # Verificar si la respuesta fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta JSON en un diccionario Python
        data = response.json()
        
        # Devolver la lista de OCIDs desde el diccionario de datos
        return data.get('data', [])
    else:
        # Si la solicitud no fue exitosa, lanzar una excepción con un mensaje de error
        raise Exception(f"La solicitud para obtener OCIDs no fue exitosa para el año {year} y la página {page}.")

# Nombre del bucket de Google Cloud Storage donde se guardarán los datos
bucket_name = os.getenv('bucket_name')

# Año que deseas recuperar
year_to_retrieve = 2022

# Número de páginas para saltar en cada iteración
page_jump = 100

# Obtener el número total de páginas para el año seleccionado
ocid_search_url = f"https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/search_ocds?page=1&year={year_to_retrieve}"
response = requests.get(ocid_search_url)

# Verificar si la respuesta fue exitosa
if response.status_code == 200:
    data = response.json()
    total_pages = data.get('pages', 0)
else:
    # Si la solicitud no fue exitosa, lanzar una excepción con un mensaje de error
    raise Exception(f"No se pudo obtener información para el año {year_to_retrieve}.")

# Iterar a través de las páginas para obtener los OCIDs y almacenarlos en GCS
page = 1
while page <= total_pages:
    # Obtener la lista de OCIDs para la página actual
    ocids = get_ocids(year_to_retrieve, page)
    
    if ocids:
        # Convertir los datos en un DataFrame de pandas
        df_ocids = pd.DataFrame(ocids)
        
        # Definir un nombre de archivo único para cada página
        file_name = f"datos/{year_to_retrieve}/ocids_{year_to_retrieve}_page_{page}.csv"
        
        # Configurar la conexión a Google Cloud Storage
        client = storage.Client()
        
        # Obtener el bucket de almacenamiento
        bucket = client.bucket(bucket_name)
        
        # Crear un objeto blob en el bucket con el nombre de archivo
        blob = bucket.blob(file_name)
        
        # Cargar el DataFrame como un archivo CSV en Google Cloud Storage
        blob.upload_from_string(df_ocids.to_csv(index=False), content_type='text/csv')
        
        # Imprimir un mensaje indicando que los OCIDs se han almacenado en GCS
        print(f"Los OCIDs para la página {page} del año {year_to_retrieve} se han almacenado en GCS: {file_name}")
    
    # Avanzar al siguiente conjunto de páginas
    page += page_jump


En resumen del este primer paso:    
- Importa las bibliotecas necesarias para realizar solicitudes HTTP, trabajar con datos tabulares y utilizar Google Cloud Storage.
- Define una función llamada get_ocids(year, page) que realiza una solicitud HTTP GET para obtener OCIDs de un año específico y una página específica desde una URL proporcionada.
- Especifica el nombre del bucket de Google Cloud Storage donde se guardarán los datos y el año que se desea recuperar.
- Obtiene el número total de páginas disponibles para el año seleccionado a través de una solicitud HTTP.
- Itera a través de las páginas, obtiene los OCIDs, los convierte en un DataFrame de pandas y los carga en Google Cloud Storage en archivos CSV. Luego, imprime un mensaje indicando el éxito de la operación.
- El ciclo continúa avanzando al siguiente conjunto de páginas hasta que se hayan procesado todas las páginas disponibles para el año especificado.

## PASO 2: CARGAR LOS DATOS A GOOGLE CLOUD BIGQUERY DESDE GOOGLE CLOUD STORAGE

Este código demuestra una solución robusta y escalable para la carga de datos en BigQuery desde archivos CSV almacenados en Google Cloud Storage (GCS). Es altamente adaptable y automatizable, lo que lo hace ideal para la recepción regular de datos actualizados, ya sea mensual, trimestral u otro período definido. La flexibilidad radica en su capacidad para crear dinámicamente tablas en BigQuery, detectar automáticamente el esquema de datos y cargar archivos CSV de manera eficiente desde GCS. Esto permite a las organizaciones mantener sus bases de datos actualizadas de manera fluida y sin problemas, adaptándose a las necesidades cambiantes de la empresa y garantizando la disponibilidad de información precisa para la toma de decisiones informadas.

In [ ]:
# Importamos las bibliotecas necesarias desde la biblioteca Google Cloud
from google.cloud import bigquery
from google.cloud.exceptions import BadRequest

# Creamos un objeto cliente de BigQuery para interactuar con Google BigQuery
client = bigquery.Client()

# Definimos una lista de años para la cual queremos realizar ciertas operaciones.
# En este caso, solo estamos utilizando el año 2015 como ejemplo.
lista_año = [2015,2016,2017,2018,2019,2020,2021,2022]

# Iteramos a través de la lista de años
for año in lista_año:
    # Definimos el identificador de la tabla en BigQuery que deseamos crear y cargar
    table_id = f"{os.getenv('table_id')}.OCID_{año}"

    # Definimos un esquema vacío para la tabla, ya que vamos a autodetectar el esquema de los datos CSV.
    schema = []

    # Creamos un objeto de tabla de BigQuery con el identificador y el esquema especificados.
    table = bigquery.Table(table_id, schema=schema)

    # Creamos la tabla en BigQuery.
    table = client.create_table(table)  # Hacemos una solicitud a la API.
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

    # Configuramos el trabajo de carga de datos en BigQuery.
    job_config = bigquery.LoadJobConfig(
        autodetect=True, source_format=bigquery.SourceFormat.CSV
    )

    # Especificamos la URI del archivo CSV que deseamos cargar desde Google Cloud Storage (GCS).
    uri = f"gs://{os.getenv('bucket_name')}/datos/{año}/*.csv"

    try:
        # Iniciamos un trabajo para cargar datos desde la URI especificada en la tabla de BigQuery.
        load_job = client.load_table_from_uri(
            uri, table_id, job_config=job_config
        )  # Hacemos una solicitud a la API.

        # Esperamos a que el trabajo de carga de datos se complete.
        load_job.result()

        # Obtenemos información sobre la tabla cargada.
        destination_table = client.get_table(table_id)

        # Imprimimos la cantidad de filas cargadas en la tabla.
        print("Loaded {} rows.".format(destination_table.num_rows))
    except BadRequest as e:
        # En caso de un error de solicitud (por ejemplo, archivo no encontrado), manejamos la excepción.
        print(f"Error loading data for year {año}: {e}")
        # Aquí podríamos tomar medidas para manejar el error, como registrarlo o saltar el archivo.
        # También se proporciona un comentario que muestra cómo eliminar el archivo de GCS en caso de error.
